<a href="https://colab.research.google.com/github/Chaddol/Bootcamp_project/blob/main/%EC%96%91%EC%B1%84%EB%A6%B0/oral_medication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### 약품식별 인공지능 개발을 위한 경구 약제 이미지 분류
##### 경구약제 약품식별 결과를 안내하고 피드백 할 수 있는 AI 서비스 모델이 요구되며 이를 위해 학습용 경구약제 이미지데이터 셋과 약품 정보를 분류 하는게 필요
##### 이미지 데이터 전처리 및 데이터 확인
##### 이미지 처리 기술을 활용하여 경구 약제 이미지 분류
#####Python, Pillow or Opencv, Deep learning
##### Git (1일 1 commit)

##### 일단 먼저 JSON 파일을 분석해보자. (데이터 살펴보기)
##### 데이터 전처리
##### 데이터 불러오기
##### input, label 만들기
##### 모델 구성
##### 학습
##### 결과 보여주기 => matplotlib 활용하여
#####        loss accuracy

In [1]:
#google drive 연결와 colab 연결하기기
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# 라이브러리 세팅팅
#import lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow
import torch
import pandas
import seaborn as sns

import glob #파일명 다 긁어오기
import PIL
import cv2 #이미지 데이터 전처리, 이미지 불러오기

import imageio #이미지 불러오기
import skimage # Numpy배열로 이미지 객체를 다룬다.
import skimage.io
import skimage.transform

from sklearn.model_selection import train_test_split

In [3]:
# Data Upload
# JSON 압축해제
# %cd /content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/
# !unzip -q -o /content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/81.zip


In [4]:
# Data Upload
# 이미지 압축해제
# %cd /content/drive/MyDrive/Colab/medicine/OriginalData/SingleMedicine/
# !unzip -q -o /content/drive/MyDrive/Colab/medicine/OriginalData/SingleMedicine/images.zip

In [5]:
# JSON 파일 분석석
import os
dir_path = "/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/"
json_files = []

for (root, directories, files) in os.walk(dir_path):
    for d in directories:
        d_path = os.path.join(root, d)
        # print(d_path)

    for file in files:
        file_path = os.path.join(root, file)
        if file_path.split('.')[-1] == 'json':
           json_files.append(file_path)
        
        # print(file_path)

print(len(json_files))

9612


In [6]:
import os
dir_path = "/content/drive/MyDrive/Colab/medicine/OriginalData/SingleMedicine/"
image_files=list()

for (root, directories, files) in os.walk(dir_path):
    for d in directories:
        d_path = os.path.join(root, d)
        # print(d_path)

    for file in files:
        file_path = os.path.join(root, file)
        image_files.append(file_path)
        # print(file_path)

print(len(image_files))


print("Number of image files: {}".format(len(image_files)))

9613
Number of image files: 9613


In [7]:
# 리스트에 dl_class_no 넣는 작업
# %cd /content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine

categories_list = list()

import json

# json 파일 프린트
from glob import glob

for filename in json_files:
  print(filename)
  if filename == '/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/81.zip':
      categories_list.append("")
      continue
  if filename == '/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-038884_json/K-038884_0_0_1_1_75_140_200.json':
    categories_list.append("")
    continue
  with open(filename, 'r', encoding="UTF-8") as f:
    json_data = json.load(f)
    # print(json_data["images"][0]["dl_class_no"])
    categories_list.append(json_data["images"][0].get('di_class_no'))

# print(json_data["images"][0].get('di_class_no'))
print(categories_list)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_040_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_080_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_120_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_100_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_060_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_140_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_160_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/SingleMedicine/K-039147_json/K-039147_0_0_0_1_75_180_200.json
/content/drive/MyDrive/Colab/medicine/LabellingData/

KeyboardInterrupt: ignored

In [ ]:
len(categories_list)


In [ ]:
image_files

In [ ]:
x_data = []
y_data = []

for p in image_files:
  print(p)
  if p == '/content/drive/MyDrive/Colab/medicine/OriginalData/SingleMedicine/images.zip':
    continue
  try:
    img = PIL.Image.open(p) # 이미지를 여기서 열고.
  except:
    continue
  #이미지 파일명 
  # 이미지 전처리 과정 https://ivo-lee.tistory.com/91
  # img_name = plt.split('/')[-1] # 019_891.png
  # img = img.convert('RGB') # RGB변환
  # img = img.resize((64,64)) # 크기 변환
  np_img = np.array(img) # 파이썬에서 배열을 사용하기 위한 표준 패키지는 넘파이(NumPy)다. 그냥 이미지를 배열로 바꿔주었다고 생각하자.
  x_data.append(np_img)
  #이미지 파일명을 통해 데이터 테이블에서 'health'의 값 읽어오기
  category = categories_list[image_files.index(p)]
  y_data.append(category) # 이로써, x_data에는 img가 y_data에는 health_status가 들어 감을 알 수 있다. 같은 index에

x_data = np.array(x_data) # 그냥 오류 나니까 하신듯?
y_data = np.array(y_data)

print(x_data.shape) # x_data의 shape
print(y_data) # y_data의 health_status
print(y_data.shape) # y_data의 shape


In [ ]:
len(categories_list)

In [ ]:
categories_set = set(categories_list)

In [ ]:
print(categories_set)

In [ ]:
# String 형태인 y_data를 학습을 위해 정수로 변환
# 학습을 위해 y_data의 string을 int로 변환했다.
categories_dict = {'':0, '[01140]해열.진통.소염제':1, '[02140]혈압강하제':2, '[02220]진해거담제':3, '[01240]진경제':4, '[01190]기타의 중추신경용약':5, '[04290]기타의 종양치료제':6, '[01490]기타의 알레르기용약':7, '[02180]동맥경화용제':8, '[02370]정장제':9, '[02380]하제, 완장제':10, '[03160]혼합비타민제(비타민AD 혼합제제를 제외)':11}

new_y_data = [] # 자 이제 정수로 변환했으니까 new_y_data로 바꾸어 주어야 겠지?

# for in 구문을 통해 변환과정 완료
for item in y_data:
  y_int = categories_dict[item]
  new_y_data.append(y_int)



In [ ]:
print(new_y_data)

In [ ]:
len(x_data)

In [ ]:
len(y_data)

In [ ]:
x_data = np.array(x_data) # 그냥 오류 나니까 하신듯?


In [ ]:
new_y_data = np.array(new_y_data) # 한번더 numpy array로 변경 완료  

print(new_y_data.shape) # 새롭게 만들어진 new_y_data가 어떤 모양인지 한 번 볼까
print(new_y_data) # new_y_data를 한 번 출력해보자

from sklearn.model_selection import train_test_split # 데이터를 나눌 때 쓰는 라이브러리인가봐
x_train, x_test, y_train, y_test = train_test_split(x_data, new_y_data, test_size = 0.2) # test_size는 0.2로 하고 나머지는 train으로  해주세요.
x_train, x_test = x_train/255.0, x_test/255.0 

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
import tensorflow.keras 

from tensorflow.keras import layers

#cnn 모델 쌓기
#Inception V3 모델 이용


#cnn 모델 쌓기
#Inception V3 모델 이용

input_img = layers.Input(shape=(64,64,3)) # input_크기: 64, 64, 3 

# 필터로 특징을 뽑아주는 convolution layer
# convolution layer는 보통 영상에 사용하나, 2D version이 따로 있는 듯
# 첫 번째 인자: convolution의 필터 수
# 두 번째 인자: convolution의 (행, 열)
# padding = 
#   'valid' 유효한 영역만 출력된다. 출력 이미지 사이즈가 입력 이미지보다 작다.
#   'same' 출력 이미지 사이즈가 입력 이미지 사이즈와 동일하다.
# activiation = 
#   'linear' 디폴트 값, 입력 뉴런과 가중치로 계산된 결과 값이 그대로 출력으로 나온다.
#   'relu' 어쩌고 저쩌고 함수
#   'sigmoid' 이진 분류 출력층에서 주로 쓰인다.
#   'softmax' 다중 클래스 분류 문제에서 주로 쓰인다.

tower_1= layers.Conv2D(64, (1,1), padding = 'same', activation = 'relu')(input_img)

tower_2= layers.Conv2D(64, (1,1), padding = 'same', activation = 'relu')(input_img)
tower_2= layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu')(tower_2)

tower_3= layers.MaxPooling2D((3,3), padding = 'same', strides = (1,1))(input_img)
tower_3= layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu')(tower_3)

tower_4= layers.Conv2D(64, (1,1), padding = 'same', activation = 'relu')(input_img)
tower_4= layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu')(tower_4)
tower_4= layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu')(tower_4)

# layers.concatenate: axis를 기준으로 배열들을 합치는 역할을 한다.
concat_layer = layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis = 3) #axis 3번째 값을 기준으로 합친다 
# layers.Flatten: 입력을 압축해준다. 배치 사이즈에는 영향을 주지 않는다.
flat_layer = layers.Flatten()(concat_layer)
# denselly connected NN layer
# 연결 신경망 층
output = layers.Dense(6, activation = 'softmax')(flat_layer) # 마지막 층: 6개 => health의 상태: 6종류

import tensorflow as tf
model = tf.keras.models.Model(input_img, output)

# 모델 구조를 모르겠을 때는 무조건,  model.summary()를 이용해서 확인하자.
model.summary() # 지금 내가 설계한 모델인 어떤지 한번 본다.

In [ ]:
# 데이터 기반 인공지능이다보니 어떻게 컴파일 할 것인지 컴퓨터에게 알려주어야 한다.
# 위에 keras를 기반으로 만든 모델에 대해서 layers를 기반으로 컴파일 해주어야 한다.
# optimizer
#   최적화 방법
#   SGD, Adam, RMSprop
#   어떠한 모델을 만드냐에 따라
# loss function
#   손실률 계산법
#   모델 최적화에 사용되는 목적함수
# metric
#   훈련을 모니터링 하기 위해서 사용됨
# 분류에서는 accuracy라고 보통 적으면 된다.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # 최적화 도구는 adam,

In [ ]:
# 위에서 컴파일까지 했으면 fit()으로 훈련시키는 것이다.
# epoch
#   전체 데이터를 epoch 번 사용해서 학습을 거친다.
#   1 Epoch: 모든 데이터셋을 한 번 학습
history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 30)
# 이 model.fit의 학습 결과를 history에 보여준다고 생각하면 된다.
# 아래의 출력물들은 epochy 과정과 loss, accuracy를 보여준다.


In [ ]:
# 위에서 학습했던 결과를 저장한 history를 accuracy와 val_accuracy를 그래프로 보여준다.
plt.plot(history.history['accuracy'], label = 'train_accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.legend()

In [ ]:
# 위에서 학습했던 결과를 저장한 history를 loss와 val_loss를 그래프로 보여준다.
plt.plot(history.history['loss'], label = 'train_loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.legend()

In [ ]:
# 시각화

In [ ]:
# train set, test set 준비
train_df, test_df = train_test_split(mediation_df, test_size=0.2, random_state=1, stratify=mediation_df['~~'])

#확인
print("Train set rows: {}".format(train_df.shape[0]))
print("Test  set rows: {}".format(test_df.shape[0]))